In [1]:
import os
import sys
import json
import inspect

sys.path.append("../../common/src")
sys.path.append("../src")
os.chdir("../src")

In [2]:
!export PROJECT_ID="gcp-mira-demo"
project = "gcp-mira-demo"
os.environ["PROJECT_ID"] = project

In [3]:
from common.models import Agent
from common.models.agent import AgentType
from services.agents.agent_service import get_agent_config, get_task_agent_config, get_plan_agent_config, get_all_agents

In [4]:
get_agent_config()

{'Chat': {'llm_type': 'VertexAI-Chat',
  'agent_type': 'langchain_Conversational',
  'agent_class': services.agents.agents.ChatAgent,
  'capabilities': ['Chat', 'Query']},
 'Task': {'llm_type': 'VertexAI-Chat',
  'agent_type': 'langchain_Conversational',
  'agent_class': services.agents.agents.TaskAgent,
  'capabilities': ['Chat', 'Query', 'Task']},
 'Plan': {'llm_type': 'OpenAI-GPT4',
  'agent_type': 'langchain_ZeroShot',
  'agent_class': services.agents.agents.PlanAgent,
  'capabilities': ['Plan']}}

In [5]:
get_task_agent_config()

{'Task': {'llm_type': 'VertexAI-Chat',
  'agent_type': 'langchain_Conversational',
  'agent_class': services.agents.agents.TaskAgent,
  'capabilities': ['Chat', 'Query', 'Task']}}

In [6]:
get_all_agents()

[{'Chat': {'llm_type': 'VertexAI-Chat', 'capabilities': ['Chat', 'Query']}},
 {'Task': {'llm_type': 'VertexAI-Chat',
   'capabilities': ['Chat', 'Query', 'Task']}},
 {'Plan': {'llm_type': 'OpenAI-GPT4', 'capabilities': ['Plan']}}]

In [7]:
from services.agents.agent_service import run_agent, agent_plan

In [8]:
run_agent("ChatAgent", "how is the weather today?")

'The weather today is expected to be sunny with a high of 75 degrees Fahrenheit. There is a slight chance of rain in the afternoon.'

In [9]:
user_id = "5nJrkPWa3D0yCKA853mD"
agent_plan("Plan", "Send an email to my boss asking for a raise", user_id)

('Task: Send an email to my boss asking for a raise\n\nThought: The task requires drafting and sending an email. This can be done using the gmail_tool. We should also consider what the user might want to include in the email such as reasons for the raise and examples of their work performance. \n\nPlan: \n   1. Use [docs_tool] to [draft the email to your boss asking for a raise, making sure to include reasons for the raise and evidence of your work performance].\n   2. Use [gmail_tool] to [send the drafted email to your boss].',
 <common.models.agent.UserPlan at 0x10bb3d5b0>)